# Effect of Alpha (α) on VAAS Anomaly Scoring

VAAS combines two different signals when assessing visual inconsistency:
- feature-level inconsistency (S_F),
- and patch-level inconsistency (S_P).

The alpha (α) parameter controls how these two signals are weighted
when computing the final hybrid score (S_H).

This notebook explores how changing α affects:
- the final anomaly score,
- and the qualitative interpretation of results.

Understanding this trade-off is important when adapting VAAS
to different practical scenarios.


## Installation

Install VAAS if it is not already present in the environment.


In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO

## Loading an Example Image

We use a single image throughout this notebook so that any changes
in scores can be attributed solely to the alpha parameter.


In [ ]:
image_url = (
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/"
    "examples/images/COCO_DF_C110B00000_00539519.jpg"
)

response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

image


## What Does Alpha Control?

Alpha determines the balance between global and local inconsistency signals:

- Lower alpha values emphasize feature-level consistency (S_F).
- Higher alpha values emphasize patch-level consistency (S_P).

Alpha does not change the underlying models or reference statistics.
It only affects how the two scores are combined.

---

## Sweeping Alpha Values

Because alpha is set at pipeline initialization time, we create
a separate VAAS pipeline for each alpha value.

This is a deliberate design choice to keep inference deterministic
and reproducible.
## Sweeping Alpha Values

Because alpha is set at pipeline initialization time, we create
a separate VAAS pipeline for each alpha value.

This is a deliberate design choice to keep inference deterministic
and reproducible.

---

In [ ]:
alpha_values = [0.0, 0.25, 0.5, 0.75, 1.0]

results = []

for alpha in alpha_values:
    pipeline = VAASPipeline.from_pretrained(
        "OBA-Research/vaas-v1-df2023",
        device="cpu",
        alpha=alpha,
    )

    output = pipeline(image)

    results.append(
        {
            "alpha": alpha,
            "S_F": output["S_F"],
            "S_P": output["S_P"],
            "S_H": output["S_H"],
        }
    )


## Comparing Scores Across Alpha Values

Below we print the scores for each alpha value to observe how the
hybrid score changes while S_F and S_P remain stable.


In [ ]:
for r in results:
    print(
        f"alpha={r['alpha']:.2f} | "
        f"S_F={r['S_F']:.4f} | "
        f"S_P={r['S_P']:.4f} | "
        f"S_H={r['S_H']:.4f}"
    )


## Key Observations

Several patterns should be apparent:

- S_F and S_P remain constant across alpha values.
- S_H shifts smoothly as alpha changes.
- At alpha = 0.0, S_H reflects only feature-level inconsistency.
- At alpha = 1.0, S_H reflects only patch-level inconsistency.

This behavior confirms that alpha acts as a weighting parameter,
not as a source of randomness or instability.

## Visualizing the Effect of Alpha

Although VAAS does not expose internal plots by default,
we can visualize how S_H varies as a function of alpha.

In [ ]:
import matplotlib.pyplot as plt

alphas = [r["alpha"] for r in results]
s_h_values = [r["S_H"] for r in results]

plt.figure(figsize=(6, 4))
plt.plot(alphas, s_h_values, marker="o")
plt.xlabel("Alpha (α)")
plt.ylabel("Hybrid Anomaly Score (S_H)")
plt.title("Effect of Alpha on VAAS Hybrid Score")
plt.grid(True)
plt.show()

## When Should Alpha Be Adjusted?

Different applications may benefit from different alpha settings:

- Lower alpha values may be preferred when detecting broad
  semantic or structural anomalies.
- Higher alpha values may be preferred when focusing on
  localized artifacts or tampering.

In many general-purpose scenarios, alpha = 0.5 provides a
reasonable balance and is a good default.

---

## Summary

In this notebook, we:
- examined the role of the alpha parameter,
- evaluated its effect on VAAS anomaly scores,
- and discussed practical guidelines for choosing alpha.

The next notebook will focus on practical considerations
when running VAAS on different hardware backends.

check it out: [05_running_vaas_on_cpu_cuda_mps.ipynb](https://colab.research.google.com/drive/1XeQjEdlWtisZoDDPp6WxwbNxoYC43wyk?usp=sharing)
